In [62]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas_ta.volatility import atr
from pandas_ta.trend import adx
import matplotlib.pyplot as plt

import plotly.graph_objects as go

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import resample, zigzag, supertrend


C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [63]:
#klines_file  = './backtest/klines/0m01/Alts_AAVEUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
#klines_file  = './backtest/klines/0m01/Alts_GMXUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
#klines_file  = './backtest/klines/0m01/Alts_GALAUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
klines_file  = './backtest/klines/0m01/Alts_AVAXUSDT_0m01_2024-05-28_2025-05-28.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)

print('Total de velas - Iniciales',len(df))
df = df[100000:300000]
df.reset_index(inplace=True, drop=True)
print('Total de velas - Ajustadas',len(df))

#df15 = resample(df,periods=15)
#df15['atr'] = atr(df15['high'],df15['low'],df15['close'],length=14)
#df15['atr_ma'] = df15['atr'].rolling(window=14).mean()
#df15['atr_rel'] = df15['atr']/df15['atr_ma'] - 1
#df15['vol_ma'] = df15['volume'].rolling(window=100).mean()
#df15['vol_rel'] = df15['volume']/df15['vol_ma'] - 1  
#df15 = supertrend(df15)
#df15 = zigzag(df15)
#
#df15.drop(columns=['vol_ma','volume','atr','atr_ma'],inplace=True)



Total de velas - Iniciales 829440
Total de velas - Ajustadas 200000


In [64]:
def get_pivots_alert(df,threshold=1.5):

    def contar_decimales(f):
        s = str(f)
        if '.' in s:
            return len(s.split('.')[1].rstrip('0'))
        return 0

    data = {}
    data['alert'] = 0
    min_periods = 50
    price = df.iloc[-1]['close']
    if df['close'].count() >= min_periods:
        
        adx_length = 14
        adx_data = adx(df['high'],df['low'],df['close'],length=adx_length)
        last_adx = adx_data.iloc[-1][f'ADX_{adx_length}']
        adx_data['adx_ma'] = adx_data[f'ADX_{adx_length}'].rolling(window=5).mean()
        last_adx_ma = adx_data.iloc[-1]['adx_ma']
                
        if last_adx > 26:
            
            df = supertrend(df)
            trend = int(df.iloc[-1]['st_trend'])
            
            if trend >= 1 or trend <= -1:

                df = zigzag(df)
                pivots = df[df['ZigZag']>0]['ZigZag'].tolist()
                
                if len(pivots) >= 5: #Se busca que existan mas pivots de lo necesario par apoder formarlos

                    df_pivots = df[df['ZigZag']>0]
                    lpdt = df_pivots.iloc[-1]['datetime'] #Last Pivot Datetime
                    max_flp = df[df['datetime']>lpdt]['high'].max() #Max From last pivot
                    min_flp = df[df['datetime']>lpdt]['low'].min()  #Min From last pivot

                    if trend >=1:
                        #Alertas en LONG

                        #Busqueda de pivots con el siguiente formato 
                        #              -1
                        #       -3
                        #                         Precio
                        #                  min_flp
                        #          -2
                        #   -4
                    
                        alert_type = 'Dual Pullback LONG'
                        last_pivots_diff = pivots[-1]-min_flp
                        valid_pivots_type = pivots[-1] > pivots[-3] and pivots[-3] > min_flp and \
                                            min_flp > pivots[-2] and pivots[-2] > pivots[-4] 
                        in_price = min_flp+last_pivots_diff/3
                        valid_price = in_price*(1-(0.25/100)) < price < in_price*(1+(0.25/100))
                        valid_pivot_delta = pivots[-1] > min_flp*(1+threshold/100) and min_flp > pivots[-2]*(1+(threshold/3)/100)
                        if valid_pivots_type and valid_price and valid_pivot_delta:

                            data['alert'] = 1
                            data['side'] = 1
                            data['alert_str'] = alert_type
                            data['sl1'] = min_flp
                            data['tp1'] = pivots[-1] 
                            data['in_price'] = data['sl1']+((data['tp1']-data['sl1'])/3) #Genera un ratio 2:1
                        
                    elif trend <= -1:
                        # Alertas en SHORT

                        #Busqueda de pivots con el siguiente formato 
                        #   -4
                        #           -2
                        #                   max_flp
                        #                              Precio
                        #       -3
                        #               -1
                        alert_type = 'Dual Pullback SHORT'
                        last_pivots_diff = max_flp-pivots[-1]
                        valid_pivots_type = pivots[-1] < pivots[-3] and pivots[-3] < max_flp and \
                                            max_flp < pivots[-2] and pivots[-2] < pivots[-4] 
                        in_price = max_flp-last_pivots_diff/3
                        valid_price = in_price*(1-(0.25/100)) < price < in_price*(1+(0.25/100))
                        valid_pivot_delta = pivots[-1] < max_flp*(1-threshold/100) and max_flp < pivots[-2]*(1-(threshold/3)/100)
                        if valid_pivots_type and valid_price and valid_pivot_delta:
                            data['alert'] = -1
                            data['side'] = -1
                            data['alert_str'] = alert_type
                            data['sl1'] = max_flp
                            data['tp1'] = pivots[-1] 
                            data['in_price'] = data['sl1']-((data['sl1']-data['tp1'])/3) #Genera un ratio 2:1


                if data['alert'] != 0:
                    decs = max(contar_decimales(data['sl1']), contar_decimales(data['tp1']))
                    data['in_price'] = round(data['in_price'],decs)
                    data['df'] = df
                    data['datetime'] = df.iloc[-1]['datetime']
                    data['last_pivot'] = lpdt
                    data['adx'] = last_adx
                    data['adx_ma'] = last_adx_ma

    return data


In [65]:

# --- Parámetros ---
MAX_15MIN_CANDLES = 200 # Número máximo de velas de 15min a mantener

# --- Inicialización ---
# DataFrame para las velas de 15 minutos que se van formando
df15_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']
df15 = pd.DataFrame(columns=df15_cols)

last_15min_candle_start_time = None


open_pos = False
alert = {}
trade_columns = ['open','side','buy','sell','close','tp','sl','adx','adx_ma']
trades = pd.DataFrame(columns=trade_columns)
trade = {}
tp = None
sl = None
total_velas = len(df)
progreso = 0
# --- Iteración sobre el DataFrame de 1 minuto ---
for row_1min in df.itertuples(index=False): # index=False para no incluir el índice de df_1min
    progreso+=1
    progreso_perc = (progreso/total_velas)*100
    print(f'Procesando: {progreso_perc:.2f}% ',len(trades),'          ',end="\r")
    current_1min_timestamp = row_1min.datetime # Accedemos a la columna datetime
    current_1min_data = row_1min # El namedtuple entero

    this_15min_slot_start_time = current_1min_timestamp.floor('15T')

    is_new_15min_candle = False
    # Comparamos con la columna 'datetime' de la última fila de df15
    if len(df15)==0 or this_15min_slot_start_time > df15['datetime'].iloc[-1]:
        is_new_15min_candle = True

    if is_new_15min_candle:
        new_15min_candle_data = {
            'datetime': this_15min_slot_start_time,
            'open': current_1min_data.open, # Acceso a campos del namedtuple
            'high': current_1min_data.high,
            'low': current_1min_data.low,
            'close': current_1min_data.close,
            'volume': current_1min_data.volume
        }
        # Añadir como una nueva fila. pd.concat es más robusto aquí.
        new_row_df = pd.DataFrame([new_15min_candle_data])
        df15 = pd.concat([df15, new_row_df], ignore_index=True) # ignore_index para mantener el índice numérico secuencial

        if len(df15) > MAX_15MIN_CANDLES:
            df15 = df15.iloc[-MAX_15MIN_CANDLES:].reset_index(drop=True) # reset_index para mantener secuencia 0..N-1

    else:
        # Actualizar la última fila de df15 usando su índice numérico (.iloc[-1] o len(df15)-1)
        last_row_index = len(df15) - 1
        df15.loc[last_row_index, 'high'] = max(df15.loc[last_row_index, 'high'], current_1min_data.high)
        df15.loc[last_row_index, 'low'] = min(df15.loc[last_row_index, 'low'], current_1min_data.low)
        df15.loc[last_row_index, 'close'] = current_1min_data.close
        df15.loc[last_row_index, 'volume'] += current_1min_data.volume
        # 'datetime' y 'open' no cambian


    if len(df15) >= MAX_15MIN_CANDLES:

        if not open_pos:
            alert = get_pivots_alert(df15[-50:],threshold=1.5)

            if alert['alert']!=0:
                open_pos = True
                trade = {'open': current_1min_timestamp,
                         'close': None,
                         'side': alert['side'],
                         'buy': row_1min.close if alert['side'] > 0 else None,
                         'sell': row_1min.close if alert['side'] < 0 else None,
                         'sl': alert['sl1'],
                         'tp': alert['tp1'],
                         'adx': alert['adx'],
                         'adx_ma': alert['adx_ma'],
                         }
                tp = alert['tp1']
                sl = alert['sl1']
                
        else: #open_pos = True
            price = row_1min.close
            high = row_1min.high
            low = row_1min.low

            if trade['side']>0: #LONG
                if high >= tp:
                    trade['sell'] = tp
                    trade['close'] = current_1min_timestamp
                    open_pos = False
                    trades.loc[len(trades)] = trade
                    trade = {}
                elif low <= sl:
                    trade['sell'] = sl
                    trade['close'] = current_1min_timestamp
                    open_pos = False
                    trades.loc[len(trades)] = trade
                    trade = {}                    
  
            else: #SHORT
                if low <= tp:
                    trade['buy'] = tp
                    trade['close'] = current_1min_timestamp
                    open_pos = False
                    trades.loc[len(trades)] = trade
                    trade = {}
                if high >= sl:
                    trade['buy'] = sl
                    trade['close'] = current_1min_timestamp
                    open_pos = False
                    trades.loc[len(trades)] = trade
                    trade = {}
      
    




In [66]:
trades['result'] = (trades['sell']/trades['buy'] - 1 ) *100
trades['adx_force'] = np.where((trades['adx']>20) & (trades['adx']>trades['adx_ma']),1,0)

long_trades = trades[trades['side']> 0 ]
short_trades = trades[trades['side']< 0 ]

tot_trades = long_trades['result'].count()
trades_win = long_trades[long_trades['result']> 0 ]['result'].count()
result = long_trades['result'].sum()

print('LONG Trades:',tot_trades)
print(' Win',round((trades_win/tot_trades)*100,2),'%')
print(' Resultado',round(result,2),'%')

tot_trades = short_trades['result'].count()
trades_win = short_trades[short_trades['result']> 0 ]['result'].count()
result = short_trades['result'].sum()

print('SHORT Trades:',tot_trades)
print(' Win',round((trades_win/tot_trades)*100,2),'%')
print(' Resultado',round(result,2),'%')

#print('Trades Positivos')
#print(trades[trades['result']>0][['adx','adx_force']].describe())
#print('Trades Negativos')
#print(trades[trades['result']<0][['adx','adx_force']].describe())

LONG Trades: 19
 Win 26.32 %
 Resultado -1.38 %
SHORT Trades: 28
 Win 25.0 %
 Resultado -0.18 %


LONG Trades: 8
 Win 37.5 %
 Resultado 1.82 %
SHORT Trades: 9
 Win 44.44 %
 Resultado 3.47 %

In [67]:
df_to_show = resample(df,periods=15)

df_to_show['sl'] = None
df_to_show['tp'] = None
df_to_show['pnl'] = None
df_to_show['price'] = df_to_show['close']
df_events = pd.DataFrame(columns=['datetime','buy','sell'])
pnl = 0
for i,trade in trades.iterrows():
    if trade['side']>0:
        df_events.loc[len(df_events)] = {'datetime': trade['open'],'buy': trade['buy'],'sell': None,}
        df_events.loc[len(df_events)] = {'datetime': trade['close'],'buy': None,'sell': trade['sell'],}
    else:
        df_events.loc[len(df_events)] = {'datetime': trade['close'],'buy': trade['buy'],'sell': None,}
        df_events.loc[len(df_events)] = {'datetime': trade['open'],'buy': None,'sell': trade['sell'],}
    pnl += trade['result']
    df_to_show['sl'] = np.where((df_to_show['datetime']>=trade['open']) & (df_to_show['datetime']<=trade['close']), trade['sl'], df_to_show['sl'])
    df_to_show['tp'] = np.where((df_to_show['datetime']>=trade['open']) & (df_to_show['datetime']<=trade['close']), trade['tp'], df_to_show['tp'])
    df_to_show['pnl'] = np.where((df_to_show['datetime']>=trade['close']) , pnl, df_to_show['pnl'])
indicators = [
     {'col': 'sl','name': 'SL','color': 'red','row': 1, 'mode':'markers',},
     {'col': 'tp','name': 'TP','color': 'green','row': 1, 'mode':'markers',},
        ]

indicators_out = [
     #{'col': 'atr_rel','name': 'ATR','color': 'yellow','row': 1, 'mode':'bars',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       {'df':df_events,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       {'df':df_events,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,show_pnl=True,indicators=indicators,indicators_out=indicators_out,events=events,) 

fig.show()